# Multi-Json

In [9]:
import json
import csv
import os
import argparse

In [12]:
results_dir = "./results"

# Reads all of the json files in the dir
def list_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

list_dir = list_json_files(results_dir)


In [13]:
def above_two(data_records):
    index=[]
    for i in range(len(data_records)):
        modules = data_records[i]["modules"].keys()
        if len(modules) > 2:
            # print(i, len(modules))
            index.append(i)
        else:
            continue
    return index

def process_files(file_paths):
    
    csv_filename = 'combined_output.csv'
    
    # Check if the output file already exists
    file_exists = os.path.isfile(csv_filename)
    # print(file_exists)
    
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        if not file_exists:
            writer.writerow(["Bin_ID", "Contig_ID", "Gene_Coordinates", "Ctg_Number", "Ctg_Tag_Id", "Mibig_biosynthetic_gene_cluster", "Mibig_Product", "Blast_name", "Blast_genecluster","Blast_annotation","Blast_perc_coverage", "Blast_perc_ident", "Blast_blastscore", "Blast_evalue", "Blast_locus_tag" ])
        
        for file_path in file_paths:

            with open(file_path, 'r') as files:
                data = json.load(files)
                index_ = above_two(data["records"])

                
                # with open(csv_filename, 'w', newline='') as csvfile:
                #     writer = csv.writer(csvfile)
                #     writer.writerow(["Bin_ID", "Contig_ID", "Gene_Coordinates", "Ctg_Number", "Ctg_Tag_Id", "Mibig_biosynthetic_gene_cluster", "Mibig_Product", "Blast_name", "Blast_genecluster","Blast_annotation","Blast_perc_coverage", "Blast_perc_ident", "Blast_blastscore", "Blast_evalue", "Blast_locus_tag" ])
                    
                for i in index_:
                    bin_id = data["input_file"]
                    contig_id = data["records"][i]["modules"]["antismash.modules.clusterblast"]["record_id"]
                    blast_info = data["records"][i]["modules"]["antismash.modules.clusterblast"]["general"]["results"][0]["ranking"][1][1]["pairings"]
                    mibig_entries = data["records"][i]["modules"]["antismash.modules.clusterblast"]["knowncluster"]["mibig_entries"]["1"]

                    # Dict for each ctg_tagid
                    blast_ = {}
                    for blast in blast_info:
                        ctg_name = blast[0].split("|")[4]
                        # ctg_cordinates = blast[0].split("|")[2]
                        blast_["ctg_cordinates"] = blast[0].split("|")[2]
                        blast_name, blast_genecluster, blast_start, blast_end, blast_strand, blast_annotation, blast_perc_ident, blast_blastscore, blast_perc_coverage, blast_evalue, blast_locus_tag  = blast[2].values()
                        blast_[ctg_name] = {"blast_name" : blast_name, 
                                            "blast_genecluster": blast_genecluster,
                                            "blast_annotation": blast_annotation,
                                            "blast_perc_coverage":blast_perc_coverage,
                                            "blast_perc_ident": blast_perc_ident,
                                            "blast_blastscore": blast_blastscore,
                                            "blast_evalue": blast_evalue,
                                            "blast_locus_tag": blast_locus_tag,
                                            "cordinates": blast[0].split("|")[2] }

                    # print(blast_)

                    for ctg_tagid, vl in mibig_entries.items():
                        ctg_id = ctg_tagid.split("_")[0]
                        mibig_entry = vl[0]
                        biosynthetic_gene_cluster_id = mibig_entry[2]
                        mibig_entries_product = mibig_entry[4]
                        blast_info_ = blast_.get(ctg_tagid)

                        if blast_info_:
                            ctg_cordinates = blast_info_["cordinates"]
                            blast_name = blast_info_["blast_name"]
                            blast_genecluster= blast_info_["blast_genecluster"]
                            blast_annotation = blast_info_["blast_annotation"]
                            blast_perc_coverage = blast_info_["blast_perc_coverage"]
                            blast_perc_ident = blast_info_["blast_perc_ident"]
                            blast_blastscore = blast_info_["blast_blastscore"]
                            blast_evalue = blast_info_["blast_evalue"]
                            blast_locus_tag = blast_info_["blast_locus_tag"]

                        else:
                            ctg_cordinates = None
                            blast_name = None
                            blast_genecluster= None
                            blast_annotation = None
                            blast_perc_coverage = None
                            blast_perc_ident = None
                            blast_blastscore = None
                            blast_evalue = None
                            blast_locus_tag = None
                            
                        writer.writerow([
                            bin_id, 
                            contig_id,
                            ctg_cordinates, 
                            ctg_id, 
                            ctg_tagid, 
                            biosynthetic_gene_cluster_id, 
                            mibig_entries_product, 
                            blast_name,
                            blast_genecluster,
                            blast_annotation,
                            blast_perc_coverage,
                            blast_perc_ident,
                            blast_blastscore,
                            blast_evalue,
                            blast_locus_tag
                        ])
            

process_files(list_dir)

In [15]:
import pandas as pd
pd.read_csv("./combined_output.csv")

,Bin_ID,Contig_ID,Gene_Coordinates,Ctg_Number,Ctg_Tag_Id,Mibig_biosynthetic_gene_cluster,Mibig_Product,Blast_name,Blast_genecluster,Blast_annotation,Blast_perc_coverage,Blast_perc_ident,Blast_blastscore,Blast_evalue,Blast_locus_tag
0,MG_323829.bin1.fa,NODE_7_length_101522_cov_12.966659,7554-8226,ctg2,ctg2_9,BGC0001040,NRP+Polyketide,SAK_RS04065,NC_007432_c715194-757390,ABC_transporter_ATP-binding_protein,100.000000,98.0,423.0,9.040000e-151,WP_000462434.1
1,MG_323829.bin1.fa,NODE_7_length_101522_cov_12.966659,25459-27655,ctg2,ctg2_25,BGC0001369,Polyketide,SAK_RS03985,NC_007432_c715194-757390,beta-ketoacyl-[acyl-carrier-protein]_synthase_...,100.000000,100.0,1434.0,0.000000e+00,WP_000118223.1
2,MG_323829.bin1.fa,NODE_7_length_101522_cov_12.966659,31472-32402,ctg2,ctg2_29,BGC0000998,NRP+Polyketide,SAK_RS03965,NC_007432_c715194-757390,ABC_transporter_ATP-binding_protein,100.000000,100.0,601.0,1.810000e-218,WP_000403526.1
3,MG_323829.bin1.fa,NODE_7_length_101522_cov_12.966659,32391-32868,ctg2,ctg2_30,BGC0001369,Polyketide,SAK_RS03960,NC_007432_c715194-757390,3-hydroxyacyl-ACP_dehydratase_FabZ,100.000000,100.0,308.0,1.760000e-107,WP_000164166.1
4,MG_323829.bin1.fa,NODE_7_length_101522_cov_12.966659,32851-33157,ctg2,ctg2_31,BGC0001369,Polyketide,SAK_RS03955,NC_007432_c715194-757390,phosphopantetheine-binding_protein,100.000000,100.0,191.0,5.940000e-63,WP_000611493.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,MG_323829.bin1.fa,NODE_125_length_11688_cov_13.230809,11096-11687,ctg47,ctg47_16,BGC0000948,Other,DX05_RS06925,NZ_CP007572_c1349890-1391168,diacylglycerol_kinase_family_lipid_kinase,100.000000,100.0,380.0,6.470000e-133,WP_000833762.1
94,MG_323757.bin1.fa,NODE_83_length_8923_cov_7.096640,NaN,ctg59,ctg59_6,BGC0000100,Polyketide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,MG_323757.bin1.fa,NODE_110_length_7016_cov_6.405832,520-1279,ctg82,ctg82_1,BGC0002667,RiPP,A5816_RS11485,NZ_NGLI01000002_c555810-572363,LytTR_family_DNA-binding_domain-containing_pro...,98.809524,93.0,460.0,1.940000e-164,WP_010733065.1
96,MG_323757.bin1.fa,NODE_110_length_7016_cov_6.405832,1280-2594,ctg82,ctg82_2,BGC0000624,RiPP,A5816_RS11480,NZ_NGLI01000002_c555810-572363,GHKL_domain-containing_protein,72.311213,88.0,527.0,6.040000e-187,WP_080114483.1


# Going Modular

In [ ]:
import json
import csv
import os
import argparse

# Reads all of the json files in the dir
def list_json_files(directory):
    json_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.json'):
                json_files.append(os.path.join(root, file))
    return json_files

list_dir = list_json_files(results_dir)

def above_two(data_records):
    index=[]
    for i in range(len(data_records)):
        modules = data_records[i]["modules"].keys()
        if len(modules) > 2:
            # print(i, len(modules))
            index.append(i)
        else:
            continue
    return index

def process_files(file_paths, csv_filename):
    
    # Check if the output file already exists
    file_exists = os.path.isfile(csv_filename)
    
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        if not file_exists:
            writer.writerow(["Bin_ID", "Contig_ID", "Gene_Coordinates", "Ctg_Number", "Ctg_Tag_Id", "Mibig_biosynthetic_gene_cluster", "Mibig_Product", "Blast_name", "Blast_genecluster","Blast_annotation","Blast_perc_coverage", "Blast_perc_ident", "Blast_blastscore", "Blast_evalue", "Blast_locus_tag" ])
        
        for file_path in file_paths:

            with open(file_path, 'r') as files:
                data = json.load(files)
                index_ = above_two(data["records"])

                
                # with open(csv_filename, 'w', newline='') as csvfile:
                #     writer = csv.writer(csvfile)
                #     writer.writerow(["Bin_ID", "Contig_ID", "Gene_Coordinates", "Ctg_Number", "Ctg_Tag_Id", "Mibig_biosynthetic_gene_cluster", "Mibig_Product", "Blast_name", "Blast_genecluster","Blast_annotation","Blast_perc_coverage", "Blast_perc_ident", "Blast_blastscore", "Blast_evalue", "Blast_locus_tag" ])
                    
                for i in index_:
                    bin_id = data["input_file"]
                    contig_id = data["records"][i]["modules"]["antismash.modules.clusterblast"]["record_id"]
                    blast_info = data["records"][i]["modules"]["antismash.modules.clusterblast"]["general"]["results"][0]["ranking"][1][1]["pairings"]
                    mibig_entries = data["records"][i]["modules"]["antismash.modules.clusterblast"]["knowncluster"]["mibig_entries"]["1"]

                    # Dict for each ctg_tagid
                    blast_ = {}
                    for blast in blast_info:
                        ctg_name = blast[0].split("|")[4]
                        # ctg_cordinates = blast[0].split("|")[2]
                        blast_["ctg_cordinates"] = blast[0].split("|")[2]
                        blast_name, blast_genecluster, blast_start, blast_end, blast_strand, blast_annotation, blast_perc_ident, blast_blastscore, blast_perc_coverage, blast_evalue, blast_locus_tag  = blast[2].values()
                        blast_[ctg_name] = {"blast_name" : blast_name, 
                                            "blast_genecluster": blast_genecluster,
                                            "blast_annotation": blast_annotation,
                                            "blast_perc_coverage":blast_perc_coverage,
                                            "blast_perc_ident": blast_perc_ident,
                                            "blast_blastscore": blast_blastscore,
                                            "blast_evalue": blast_evalue,
                                            "blast_locus_tag": blast_locus_tag,
                                            "cordinates": blast[0].split("|")[2] }

                    # print(blast_)

                    for ctg_tagid, vl in mibig_entries.items():
                        ctg_id = ctg_tagid.split("_")[0]
                        mibig_entry = vl[0]
                        biosynthetic_gene_cluster_id = mibig_entry[2]
                        mibig_entries_product = mibig_entry[4]
                        blast_info_ = blast_.get(ctg_tagid)

                        if blast_info_:
                            ctg_cordinates = blast_info_["cordinates"]
                            blast_name = blast_info_["blast_name"]
                            blast_genecluster= blast_info_["blast_genecluster"]
                            blast_annotation = blast_info_["blast_annotation"]
                            blast_perc_coverage = blast_info_["blast_perc_coverage"]
                            blast_perc_ident = blast_info_["blast_perc_ident"]
                            blast_blastscore = blast_info_["blast_blastscore"]
                            blast_evalue = blast_info_["blast_evalue"]
                            blast_locus_tag = blast_info_["blast_locus_tag"]

                        else:
                            ctg_cordinates = None
                            blast_name = None
                            blast_genecluster= None
                            blast_annotation = None
                            blast_perc_coverage = None
                            blast_perc_ident = None
                            blast_blastscore = None
                            blast_evalue = None
                            blast_locus_tag = None
                            
                        writer.writerow([
                            bin_id, 
                            contig_id,
                            ctg_cordinates, 
                            ctg_id, 
                            ctg_tagid, 
                            biosynthetic_gene_cluster_id, 
                            mibig_entries_product, 
                            blast_name,
                            blast_genecluster,
                            blast_annotation,
                            blast_perc_coverage,
                            blast_perc_ident,
                            blast_blastscore,
                            blast_evalue,
                            blast_locus_tag
                        ])
            
def main():
    parser = argparse.ArgumentParser(description="Process JSON files and output combined results to a CSV file.")
    parser.add_argument('result_dir', help='JSON files to process')
    parser.add_argument('-o', '--output', default='combined_output.csv', help='Output CSV file name (default: combined_output.csv)')
    
    args = parser.parse_args()
    
    list_jsons = list_json_files(args.result_dir)

    process_files(list_jsons, args.output)
    print(f"Output written to: {args.output}")

if __name__ == "__main__":
    main()